Note: Responses from local models can be quite slow, especially with 8-bit quantization.

With 4bit quantization, llama2-7b-chat uses about 8GB of VRAM

In [ ]:
!pip install llama-index transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.9/866.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Setup

### Data

In [ ]:
from llama_index.readers import BeautifulSoupWebReader

url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"

documents = BeautifulSoupWebReader().load_data([url])

### LLM

This should run on a T4 GPU in the free tier

In [ ]:
# huggingface api token for downloading llama2
hf_token = "hf_..."

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Index Setup

In [ ]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
from llama_index import SummaryIndex

summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

### Helpful Imports / Logging

In [ ]:
from llama_index.response.notebook_utils import display_response

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Basic Query Engine

### Compact (default)

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

**`Final Response:`** OpenAI and Meta differ on AI tools in the following ways:
OpenAI:
* OpenAI is focused on developing AI tools that are useful for lots of things, such as generating text, images, and voices.
* OpenAI's latest updates include adding a voice feature to its large language model, allowing users to interact with it via voice.
* OpenAI's AI tools are designed to be more personalized, empathetic, and available.
* OpenAI's AI voice feature is still in its early stages and is only available to ChatGPT Plus subscribers.
* OpenAI's AI tools are not necessarily designed to entertain or create a synthetic social network.

Meta:
* Meta is focused on developing AI tools that can be used in its messaging apps, such as generating stickers and placing AI characters on every major surface of its products.
* Meta's AI tools are designed to create a partially synthetic social network, where AI characters will be placed on every major surface of its products.
* Meta's AI tools are designed to be more personalized, engaging, and entertaining.
* Meta's A

### Refine

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

**`Final Response:`** Based on the new context provided, OpenAI and Meta differ in their approach to AI tools in the following ways:
1. Focus: OpenAI tends to focus on developing language models, while Meta is building LLMs and chatbots for its messaging apps with a focus on entertainment and personality-driven chatbots.
2. Personality: OpenAI's chatbot has a dry, sterile text unadorned by any hint of style, while Meta's chatbots are personality-driven and have 28 different personalities.
3. Availability: The voice feature of ChatGPT is only available to ChatGPT Plus subscribers, while Meta's chatbots are available in its messaging apps.
Overall, OpenAI and Meta have different approaches to developing AI tools, with OpenAI focusing on language models and efficiency, while Meta is more focused on entertainment and personality-driven chatbots.

### Tree Summarize

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

**`Final Response:`** OpenAI and Meta have different approaches to AI tools. OpenAI focuses on developing AI language models that can be used for various tasks such as text generation, voice interactions, and image creation. They have also created chatbots that can be used as virtual assistants. On the other hand, Meta is focusing on creating AI characters that can be used in their messaging apps. They have partnered with celebrities to create personalities for their chatbots. OpenAI tends to present its products as productivity tools, while Meta is in the entertainment business and is building LLMs. OpenAI's AI tools are more focused on practical applications, while Meta's AI tools are more geared towards entertainment and personalization.

## Router Query Engine

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

### Single Selector

In [ ]:
from llama_index.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    select_multi=False
)

response = query_engine.query("What was mentioned about Meta?")

display_response(response)

ValueError: ignored

### Multi Selector

In [ ]:
from llama_index.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    select_multi=True,
)

response = query_engine.query("What was mentioned about Meta? Summarize with any other companies mentioned in the entire document.")

display_response(response)

ValueError: ignored

## SubQuestion Query Engine

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    verbose=True,
)

response = query_engine.query("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")

display_response(response)

OutputParserException: ignored

## SQL Query Engine

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!curl https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O /content/chinook.zip
!unzip /content/chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  1334k      0 --:--:-- --:--:-- --:--:-- 1338k
curl: (3) URL using bad/illegal format or missing URL
Archive:  /content/chinook.zip
  inflating: chinook.db              


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:////content/chinook.db")

In [ ]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    service_context=service_context
)

In [ ]:
response = query_engine.query("What are some albums?")

display_response(response)

NotImplementedError: ignored

In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

display_response(response)

NotImplementedError: ignored

This last query should be a more complex join

In [ ]:
response = query_engine.query("What are some tracks from the artist AC/DC? Limit it to 3")

display_response(response)

NotImplementedError: ignored

In [ ]:
print(response.metadata['sql_query'])

## Programs

Depending the LLM, you will have to test with either `OpenAIPydanticProgram` or `LLMTextCompletionProgram`

In [ ]:
from typing import List
from pydantic import BaseModel

from llama_index.program import OpenAIPydanticProgram, LLMTextCompletionProgram

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [ ]:
from llama_index.output_parsers import PydanticOutputParser

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Album),
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=True,
)

This seems to error out only because it ran out of output token space. We could fix this by setting `max_new_tokens` on the constructor higher than the default of 256.

In [ ]:
output = program(movie_name="The Shining")

ValidationError: ignored

In [ ]:
print(output)

## Data Agent

Similar to programs, OpenAI LLMs will use `OpenAIAgent`, while other LLMs will use `ReActAgent`.

In [ ]:
from llama_index.agent import OpenAIAgent, ReActAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

In [ ]:
response = agent.chat("Hello!")
print(response)

Response: Great, I'm happy to help you with your question! Can you please provide more context or clarify what you need help with? For example, what kind of information are you trying to find or what question are you trying to answer?
Great, I'm happy to help you with your question! Can you please provide more context or clarify what you need help with? For example, what kind of information are you trying to find or what question are you trying to answer?


Interesting tool inputs and responses, but I guess it works lol

In [ ]:
response = agent.chat("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")
print(response)

Response: Great, let's get started! Based on your question, I understand that you want to know the difference between Meta and OpenAI. To answer this question, I will use the `summary` tool to summarize some relevant information about both Meta and OpenAI.
Action: summary
Action Input: {"text": "Meta and OpenAI are both AI research organizations, but they have some key differences. Meta was founded in 2015 by Mark Zuckerberg and is focused on developing AI technologies for Facebook and other Facebook-owned platforms. OpenAI, on the other hand, was founded in 2015 by a group of prominent AI researchers and is focused on developing AI technologies for a wide range of applications, including but not limited to natural language processing, computer vision, and robotics. Additionally, OpenAI is a non-profit organization, while Meta is a for-profit company. Both organizations have made significant contributions to the field of AI and have published numerous research papers and articles. Howe